In [0]:
# !git clone https://github.com/rr2n17/AdvML.git

In [34]:
!ls

AdvML  datalab


## Install libraries

In [0]:
# !pip install --upgrade pip
# !pip install nltk
# !apt-get install libenchant1c2a
# !pip install pyenchant
# !pip install joblib textblob -U

In [0]:
import pandas as pd
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data
from nltk.tokenize import TreebankWordTokenizer
import re
import enchant
from enchant.checker import SpellChecker
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated

In [0]:
# nltk.download("averaged_perceptron_tagger")
# nltk.download("wordnet")
# nltk.download('punkt')
# nltk.download('stopwords')

## Read data

In [129]:
path = "AdvML/Datasets/rawData/train.csv"
dataset = pd.read_csv(path)
dataset.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## Upsample

In [0]:
ds = dataset[dataset['toxic'] == 1]
ds = ds[['comment_text', 'toxic']][70:110]

tokenizer = TreebankWordTokenizer()
chkr = SpellChecker("en_UK", "en_US")

def low(x):
    # spelling checker
    x = re.sub(r'[\W_]+', ' ', x)
    chkr.set_text(x)
    for err in chkr:
        if len(err.suggest()) > 0:
            sug = err.suggest()[0]
            err.replace(sug)
            x = chkr.get_text()
    
    return x.lower()


ds['comment_text'] = ds['comment_text'].apply(lambda x: low(x))

In [0]:
def translate(comment, language):
    if hasattr(comment, "decode"):
        comment = comment.decode("utf-8")

    text = TextBlob(comment)
    try:
#         text = text.translate(to='de')
        text = text.translate(to=language)
        text = text.translate(to="en")
    except NotTranslated:
        pass

    return str(text)

In [0]:
dp = ds[["comment_text", "toxic"]].copy()
df = ds[["comment_text", "toxic"]].copy()

i = 0

def upsample(comment, label):
    global dp, i
    l = ['es', 'fr', 'it']
    for language in l:
        new_comment = translate(comment, language)
        dp = dp.append(pd.Series({"comment_text":new_comment, "toxic":label}), ignore_index=True)
    print(i)
    i += 1
        
df.apply(lambda x: upsample(x["comment_text"], x["toxic"]), axis=1)
print(dp.head())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [0]:
# Load the pretrained neural net
tokenizer = nltk.data.load('../../../nltk_data/tokenizers/punkt/english.pickle')


for text in dp["comment_text"]:
    output=""
    # Tokenize the text
    tokenized = tokenizer.tokenize(text)

    # Get the list of words from the entire text
    words = word_tokenize(text)

    # Identify the parts of speech
    tagged = nltk.pos_tag(words)

    for i in range(0,len(words)):
        
        replacements = []

        # Only replace nouns with nouns, vowels with vowels etc.
        for syn in wordnet.synsets(words[i]):

            # Do not attempt to replace proper nouns or determiners
            if tagged[i][1] == 'NNP' or tagged[i][1] == 'DT':
                break

            # The tokenizer returns strings like NNP, VBP etc
            # but the wordnet synonyms has tags like .n.
            # So we extract the first character from NNP ie n
            # then we check if the dictionary word has a .n. or not 
            word_type = tagged[i][1][0].lower()
            if syn.name().find("."+word_type+"."):
                # extract the word only
                r = syn.name()[0:syn.name().find(".")]
                replacements.append(r)

        if len(replacements) > 0:
            # Choose a random replacement
            replacement = replacements[randint(0,len(replacements)-1)]
            output = output + " " + replacement
        else:
            # If no replacement could be found, then just use the
            # original word
            output = output + " " + words[i]
    
    dp = dp.append(pd.Series({"comment_text":output,"toxic":1}),ignore_index=True)

In [0]:
def tok_and_rem(x):
    s = x.split()
    l = list(s)
    for word in s:
        if (len(word) <= 2):
            l.remove(word)
        elif word in stopwords.words('english'):
            l.remove(word)
    return l

dp['comment_text'] = dp['comment_text'].apply(lambda x: tok_and_rem(x))

In [118]:
dp.head()

,comment_text,toxic
0,"[cocksucker, piss, around, work]",1
1,"[hey, talk, exclusive, group, tali, bans, good...",1
2,"[bye, look, come, think, coming, back, tosser]",1
3,"[gay, antisemitic, archangel, white, tiger, me...",1
4,"[fuck, filthy, mother, ass, dry]",1
